In [ ]:
## Analyzing Social Networks using GraphX/GraphFrame
## Add this maven library to cluster to be able to use graphframes - graphframes:0.8.2-spark3.1-s_2.12

vertices = spark.read.option("header","true").csv("/FileStore/tables/musae_git_target.csv")

edges = spark.read.option("header","true").csv("/FileStore/tables/musae_git_edges.csv")

In [ ]:
vertices = vertices.select("id", "name")
vertices.collect()

Out[3]: [Row(id='0', name='Eiryyy'),
 Row(id='1', name='shawflying'),
 Row(id='2', name='JpMCarrilho'),
 Row(id='3', name='SuhwanCha'),
 Row(id='4', name='sunilangadi2'),
 Row(id='5', name='j6montoya'),
 Row(id='6', name='sfate'),
 Row(id='7', name='amituuush'),
 Row(id='8', name='mauroherlein'),
 Row(id='9', name='ParadoxZero'),
 Row(id='10', name='llazzaro'),
 Row(id='11', name='beeva-manueldepaz'),
 Row(id='12', name='damianmuti'),
 Row(id='13', name='apobbati'),
 Row(id='14', name='hwlv'),
 Row(id='15', name='haroldoramirez'),
 Row(id='16', name='jasonblanchard'),
 Row(id='17', name='BahiHussein'),
 Row(id='18', name='itsmevanessi'),
 Row(id='19', name='nwjsmith'),
 Row(id='20', name='chengzhipeng'),
 Row(id='21', name='tiensonqin'),
 Row(id='22', name='pdokas'),
 Row(id='23', name='maxfierke'),
 Row(id='24', name='davidthewatson'),
 Row(id='25', name='kidbai'),
 Row(id='26', name='Micanss'),
 Row(id='27', name='hepin1989'),
 Row(id='28', name='ashphy'),
 Row(id='29', name='soberan

In [ ]:
edges = edges.withColumnRenamed("id_1", "src").withColumnRenamed("id_2", "dst")
edges.collect()

Out[4]: [Row(src='0', dst='23977'),
 Row(src='1', dst='34526'),
 Row(src='1', dst='2370'),
 Row(src='1', dst='14683'),
 Row(src='1', dst='29982'),
 Row(src='1', dst='21142'),
 Row(src='1', dst='20363'),
 Row(src='1', dst='23830'),
 Row(src='1', dst='34035'),
 Row(src='6067', dst='19720'),
 Row(src='6067', dst='20183'),
 Row(src='3', dst='4950'),
 Row(src='3', dst='18029'),
 Row(src='3', dst='3358'),
 Row(src='3', dst='34935'),
 Row(src='3', dst='5916'),
 Row(src='4', dst='2865'),
 Row(src='4', dst='9342'),
 Row(src='5', dst='27803'),
 Row(src='6', dst='27803'),
 Row(src='6', dst='18612'),
 Row(src='6', dst='18876'),
 Row(src='6', dst='31890'),
 Row(src='6', dst='17647'),
 Row(src='6', dst='18562'),
 Row(src='7', dst='37493'),
 Row(src='7', dst='33643'),
 Row(src='7', dst='30199'),
 Row(src='7', dst='35773'),
 Row(src='7', dst='11273'),
 Row(src='7', dst='12775'),
 Row(src='7', dst='21864'),
 Row(src='7', dst='9051'),
 Row(src='34957', dst='21395'),
 Row(src='34957', dst='9891'),
 Row(s

In [ ]:
from graphframes import GraphFrame
gitGraph = GraphFrame(vertices, edges)
outDegreeDF = gitGraph.outDegrees
## Top 5 nodes with highest outDegree with count of outgoing edges in each
outDegreeDF.sort(['outDegree'], ascending=[0]).limit(5).show()

+-----+---------+
|   id|outDegree|
+-----+---------+
|27803|     6809|
|31890|     1988|
|13638|     1610|
|19222|     1459|
| 9051|     1378|
+-----+---------+



In [ ]:
inDegreeDF = gitGraph.inDegrees
## Top 5 nodes with highest inDegree with count of outgoing edges in each
inDegreeDF.sort(['inDegree'], ascending=[0]).limit(5).show()

+-----+--------+
|   id|inDegree|
+-----+--------+
|31890|    7470|
|35773|    2401|
|36652|    2285|
|18163|    1858|
|19222|    1499|
+-----+--------+



In [ ]:
## Calculate PageRank for each node and output 5 nodes with the highest PageRank values
PageRankCalculation = gitGraph.pageRank(resetProbability=0.15, tol=0.01)
PageRankCalculation.vertices.sort(['pagerank'],ascending=[0]).limit(5).show()

+-----+--------------+------------------+
|   id|          name|          pagerank|
+-----+--------------+------------------+
|31890|  dalinhuang99| 746.3836811583072|
|36652|       rfthusn|497.07802333180797|
|18163|      nelsonic|399.81195289415604|
|36628|SuriyaaKudoIsc|354.96392511545287|
|35008|           mdo|  198.838173606271|
+-----+--------------+------------------+



In [ ]:
## Run connected components algorithm and find top 5 components with largest number of nodes 
sc.setCheckpointDir("/tmp/graphframes-connected-components")
minGraph = GraphFrame(vertices, edges.sample(False, 0.1))
connectedComponentsResult = minGraph.connectedComponents()
ccDF = connectedComponentsResult.rdd.map(lambda x : (x[2], 1)).reduceByKey(lambda x,y: x + y)
display(ccDF.sortBy(lambda x : -x[1]).toDF(["component", "node_count"]).take(5))


component,node_count
1,16669
94489280632,7
128849019049,7
910533066752,6
60129542202,5


In [ ]:
## Run triangle counts algorithm on each vertice and output top 5 vertices with largest triangle count.
from pyspark.sql import functions as F
triangleResults = gitGraph.triangleCount()
display(triangleResults.select("id", "count").sort(F.col("count").desc()).take(5))

id,count
27803,80286
31890,63205
19222,27873
35773,22758
13638,19762
